Extraction of audios from MELD database videos ("dev_splits_complete" folder)



In [ ]:
import subprocess
import os
import re
mkdir "meld" #Folder where the audios will be transferred
path = "meld"  
folder = "dev_splits_complete"

for video in os.listdir(folder):
  k = folder+'/'+video
  
  begin_file = re.split("\.", video)

  j = path+'/'+begin_file[0]+'.wav'
  #print(j)


  !ffmpeg -i $k -ab 160k -ac 2 -ar 44100 -vn $j
  

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf56.40.101
  Duration: 00:00:02.77, start: 0.000000, bitrate: 2140 kb/s
    Chapter #0:0: start 0.000000, end 2.751000
    Metadata:
      title           : 00:09:41.498
    Stream #0:0(eng): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 1280x720 [SAR 1:1 DAR 16:9], 2015 kb/s, 23.98 fps, 23.98 tbr, 27021 tbn, 47.95 tbc (default)
    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(eng): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, 5.1, fltp, 128 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
    Stream #0:2(eng): Data: bin_data (text / 0x74786574), 0 kb/s
    Metadata:
      handler_name    : SubtitleHandler
Stream mapping:
  Stream #0:1 -> #0:0 (aac (native) -> pcm_s16le (native))
Press [q] to stop, [?] for help
Output #0, wav, to 'audio_dev/dia4_utt0.wav':
  Metadata:
    major_brand     : isom
    minor_v

Labelling audios after Normalisation and Noise reduction

In [ ]:
import pandas as pd
import os
parent_dir = "meld"
def prepare_datadf(parent_dir): # a function whose parameter is the audio folder
    df = pd.DataFrame(columns = ['audio']) #dataframe columns
    
    for  fichier_audio in os.listdir(parent_dir):
      # for each element in the audio folder

     
        
      if fichier_audio.endswith('outNoise.wav'):
                    
        df = df.append(pd.DataFrame({'audio':[fichier_audio]}), 
                           ignore_index=True) # here at df defined, with the columns we add the values:
                                            #the audio column will take the audios_path, 
                                            #and the emotion column will take the corresponding emotion, ie the name of the folder
    #Shuffling for randomness
    df = df.sample(frac=1.0).reset_index(drop=True)
    return df
datadf = prepare_datadf(parent_dir) #function call
display(datadf.head()) #dataframe display



,audio
0,dia72_utt8_norm_outNoise.wav
1,dia30_utt5_norm_outNoise.wav
2,dia69_utt18_norm_outNoise.wav
3,dia31_utt1_norm_outNoise.wav
4,dia108_utt9_norm_outNoise.wav


In [ ]:
datacsv = pd.read_csv("dev_sent_emo.csv", encoding="utf8")
display(datacsv)
print(len(datacsv))

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,"Oh my God, hes lost it. Hes totally lost it.",Phoebe,sadness,negative,0,0,4,7,"00:20:57,256","00:21:00,049"
1,2,What?,Monica,surprise,negative,0,1,4,7,"00:21:01,927","00:21:03,261"
2,3,"Or! Or, we could go to the bank, close our acc...",Ross,neutral,neutral,1,0,4,4,"00:12:24,660","00:12:30,915"
3,4,Youre a genius!,Chandler,joy,positive,1,1,4,4,"00:12:32,334","00:12:33,960"
4,5,"Aww, man, now we wont be bank buddies!",Joey,sadness,negative,1,2,4,4,"00:12:34,211","00:12:37,505"
...,...,...,...,...,...,...,...,...,...,...,...
1104,1174,No.,Monica,sadness,negative,113,9,6,2,"00:19:28,792","00:19:29,876"
1105,1175,What? Oh my God! Im gonna miss you so much!,Rachel,sadness,negative,113,10,6,2,"00:19:33,213","00:19:35,965"
1106,1176,Im gonna miss you!,Monica,sadness,negative,113,11,6,2,"00:19:36,175","00:19:37,967"
1107,1177,I mean its the end of an era!,Rachel,sadness,negative,113,12,6,2,"00:19:39,094","00:19:40,928"


1109


Association of each audio with its emotion

In [ ]:
import pandas as pd
import os

df = pd.DataFrame(columns = ['audio','dialogue', 'utterance', 'emotion']) #dataframe columns
array = datadf.values
audios = array[:,0]

    
for  fichier_audio in audios:
  sep = re.split("\_", fichier_audio)
  dia = sep[0][3:]
  dia = int(dia)
  utt = sep[1][3:]
  utt = int(utt)
  #print(dia, utt)
  for index, row in datacsv.iterrows():
    list = [row['Dialogue_ID'], row['Utterance_ID'], row['Emotion']]
    #print(list)


    if dia == list[0] and utt == list[1]:
      df = df.append(pd.DataFrame({'audio':[fichier_audio], 'dialogue' : [list[0]], 'utterance' : [list[1]], 'emotion' : [list[2]]}), 
                          ignore_index=True) # here at df defined, with the columns we add the values:
                                            #the audio column will take the audios_path, 
                                            #and the emotion column will take the corresponding emotion, ie the name of the folder

display(df)



,audio,dialogue,utterance,emotion
0,dia72_utt8_norm_outNoise.wav,72,8,neutral
1,dia30_utt5_norm_outNoise.wav,30,5,surprise
2,dia69_utt18_norm_outNoise.wav,69,18,surprise
3,dia31_utt1_norm_outNoise.wav,31,1,surprise
4,dia108_utt9_norm_outNoise.wav,108,9,neutral
...,...,...,...,...
1086,dia32_utt9_norm_outNoise.wav,32,9,anger
1087,dia40_utt4_norm_outNoise.wav,40,4,surprise
1088,dia9_utt7_norm_outNoise.wav,9,7,neutral
1089,dia95_utt10_norm_outNoise.wav,95,10,neutral
